In [35]:
import pydicom
import pynetdicom
from pynetdicom import AE
from pydicom import uid

## Send a C-echo request to a verification SCP

Verification SOP Class has a UID of:  1.2.840.10008.1.1

we can use the UID str directly when adding the requested presentation context

In [31]:
addr = "www.dicomserver.co.uk"
port = 104 # 11112
ae_title="AET"

# <span style='color:green'> Supported DIMSE Services </span>
## <span style='color:orange'> SCU Services </span>
When the AE is acting as an SCU and an association has been established with a peer SCP, the following DIMSE-C and -N services are available:


| DIMSE service	| Association method |
|---	|---|
| C-ECHO	| Association.send_c_echo() |
| C-FIND	| Association.send_c_find(dataset, query_model) |
| C-GET	| Association.send_c_get(dataset, query_model) |
| C-MOVE	| Association.send_c_move(dataset, move_aet, query_model) |
| C-STORE	| Association.send_c_store(dataset) |
| N-ACTION	| Association.send_n_action(dataset, action_type, class_uid, instance_uid) |
| N-CREATE	| Association.send_n_create(dataset, class_uid, instance_uid) |
| N-DELETE	| Association.send_n_delete(class_uid, instance_uid) |
| N-EVENT-REPORT	| Association.send_n_event_report(dataset, event_type, class_uid, instance_uid) |
| N-GET	| Association.send_n_get(identifier_list, class_uid, instance_uid) |
| N-SET	| Association.send_n_set(dataset, class_uid, instance_uid) |

In [46]:
uid_verification_sop_class = '1.2.840.10008.1.1'


# Associate with a peer AE at IP
ae = AE(ae_title=ae_title)

ae.add_requested_context(uid_verification_sop_class)

associate = ae.associate(addr=addr, port=port)
print('associate is' + ('' if associate.is_established else 'not') + ' established')

# Example: Send a DIMSE C-ECHO request to the peer AE </span>
status = associate.send_c_echo()
print(status)

# associate.release()

associate is established
(0000, 0900) Status                              US: 0


```python
AE().associate.send_c_get(dataset: Dataset, query_model: Union[str, UID], msg_id: int = 1, priority: int = 2)→ Iterator[Tuple[Dataset, Optional[Dataset]]][source]¶

```

Parameters
- `dataset` (`pydicom.dataset.Dataset`) – The `C-GET` request’s Identifier dataset. The exact requirements for the Identifier are Service Class specific (see the DICOM Standard, Part 4).

- `query_model` (`pydicom.uid.UID` or `str`) – The value to use for the `C-GET` request’s `(0000,0002)` Affected SOP Class UID parameter, which usually corresponds to the Information Model that is to be used.

- `msg_id` (`int`, `optional`) – The `C-GET` request’s Message ID, must be between `0` and `65535`, inclusive, (**`default 1`**).

- `priority` (`int`, `optional`) – The `C-GET` request’s Priority parameter, must be `0` (`Medium`), `1` (`High`) or `2` (`Low`) (**`default 2`**)




```bash
python -m pynetdicom findscu www.dicomserver.co.uk 104 -k QueryRetrieveLevel=STUDY

```

In [57]:
from pynetdicom import sop_class


# Associate with a peer AE at IP
ae = AE(ae_title=ae_title)
ae.add_requested_context(sop_class.PatientRootQueryRetrieveInformationModelFind)

ds = pydicom.dataset.Dataset()
ds.QueryRetrieveLevel = 'STUDY' # 'PATIENT', 'STUDY', 'SERIES', 'IMAGE'


assoc = ae.associate(addr=addr, port=port)
if assoc.is_established:
    # Send the C-FIND request
    responses = assoc.send_c_find(dataset=ds, query_model=sop_class.PatientRootQueryRetrieveInformationModelFind)
    
    if 0:
        for (status, identifier) in responses:
            if status:
                print('C-FIND query status: 0x{0:04X}'.format(status.Status))
            else:
                print('Connection timed out, was aborted or received invalid response')

        # Release the association
        assoc.release()
    
else:
    print('Association rejected, aborted or never connected')
    
    
next(responses)

((0000, 0900) Status                              US: 65280,
 (0008, 0005) Specific Character Set              CS: ''
 (0008, 0052) Query/Retrieve Level                CS: 'STUDY'
 (0008, 0054) Retrieve AE Title                   AE: 'ANY-SCP')

In [54]:
from pydicom.dataset import Dataset

from pynetdicom import AE, evt, build_role, debug_logger


debug_logger()

# Implement the handler for evt.EVT_C_STORE
def handle_store(event):
    """Handle a C-STORE request event."""
    ds = event.dataset
    ds.file_meta = event.file_meta

    # Save the dataset using the SOP Instance UID as the filename
    ds.save_as(ds.SOPInstanceUID, write_like_original=False)

    # Return a 'Success' status
    return 0x0000

handlers = [(evt.EVT_C_STORE, handle_store)]

# Initialise the Application Entity
ae = AE()

# Add the requested presentation contexts (QR SCU)
ae.add_requested_context(sop_class.PatientRootQueryRetrieveInformationModelGet)

# Add the requested presentation context (Storage SCP)
ae.add_requested_context(sop_class.CTImageStorage)

# Create an SCP/SCU Role Selection Negotiation item for CT Image Storage
role = build_role(sop_class.CTImageStorage, scp_role=True)

# Create our Identifier (query) dataset
# We need to supply a Unique Key Attribute for each level above the
#   Query/Retrieve level
ds = Dataset()
ds.QueryRetrieveLevel = 'SERIES'

# Unique key for PATIENT level
ds.PatientID = '1234567'

# Unique key for STUDY level
ds.StudyInstanceUID = '1.2.3'

# Unique key for SERIES level
ds.SeriesInstanceUID = '1.2.3.4'

# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate("127.0.0.1", 11112, ext_neg=[role], evt_handlers=handlers)

if assoc.is_established:
    # Use the C-GET service to send the identifier
    responses = assoc.send_c_get(ds, sop_class.PatientRootQueryRetrieveInformationModelGet)
    
    for (status, identifier) in responses:
        if status:
            print('C-GET query status: 0x{0:04x}'.format(status.Status))
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

((0000, 0900) Status                              US: 65280,
 (0008, 0005) Specific Character Set              CS: ''
 (0008, 0052) Query/Retrieve Level                CS: 'STUDY'
 (0008, 0054) Retrieve AE Title                   AE: 'ANY-SCP')

In [49]:
from dataclasses import dataclass

# from pynetdicom.sop_class import VerificationSOPClass
from pynetdicom.sop_class import VerificationServiceClass, Verification


@dataclass
class Modality:
    addr: str
    port: int
    ae_title: str

class Association:
    def __init__(self, modality, context):
        self.modality = modality
        self.context = context

    def __enter__(self):
        ae = AE()
        ae.add_requested_context(self.context)
        self._association = ae.associate(**vars(self.modality))
        return self._association

    def __exit__(self, *args):
        self._association.release()
        self._association = None
        
        
modality = Modality('www.dicomserver.co.uk', 104, 'AET')


if __name__ == '__main__':
    modality = Modality('www.dicomserver.co.uk', 104, 'AET')

    with Association(modality, Verification) as assoc:
        resp = assoc.send_c_echo()
        # assoc.req
        print(f'Modality responded with status: {resp.Status}')
        
        # request the list of studies
        resp = assoc.send_c_find()
        # for (status, dataset) in resp:
        #     print(f'Modality responded with status: {status}')
        #     if dataset:
        #         print(dataset)


Requesting Association

Association Accepted

Sending Echo Request: MsgID 1

Received Echo Response (Status: 0x0000 - Success)

Modality responded with status: 0


Releasing Association

TypeError: Association.send_c_find() missing 2 required positional arguments: 'dataset' and 'query_model'